In [82]:
import cv2
import os
from collections import defaultdict

In [83]:
# ORB detector and matcher
orb = cv2.ORB_create(nfeatures=1000)
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Step 1: Load descriptors for all people
people_descriptors = defaultdict(list)

ref_base_path = 'images/reference_faces/'
for person in os.listdir(ref_base_path):
    person_path = os.path.join(ref_base_path, person)
    if not os.path.isdir(person_path):
        continue
    for img_name in os.listdir(person_path):
        img_path = os.path.join(person_path, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            kp, des = orb.detectAndCompute(img, None)
            if des is not None:
                people_descriptors[person].append(des)

In [84]:
# Step 2: Process video
cap = cv2.VideoCapture('images/indentify.mp4')
if not cap.isOpened():
    raise Exception("Could not open video")


# Setup writer
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_videos/output_detected.mp4', fourcc, fps, (width, height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_roi = gray[y:y+h, x:x+w]
        kp, des = orb.detectAndCompute(face_roi, None)

        if des is None:
            continue

        best_match_name = None
        best_avg_distance = float('inf')
        threshold = 40  # ORB threshold for good match

        for person, descriptors_list in people_descriptors.items():
            for ref_des in descriptors_list:
                matches = bf.match(ref_des, des)
                if len(matches) >= 10:
                    matches = sorted(matches, key=lambda m: m.distance)
                    avg_dist = sum(m.distance for m in matches[:10]) / 10
                    if avg_dist < best_avg_distance:
                        best_avg_distance = avg_dist
                        best_match_name = person

        if best_avg_distance < threshold and best_match_name is not None:
            # print(f"{best_match_name} detected with avg dist: {best_avg_distance:.2f}")
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            if best_match_name == 'ronaldo':
                best_match_name = 'GOAT'
            cv2.putText(frame, best_match_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.9, (0, 255, 0), 2)
    out.write(frame)
    cv2.imshow("Multi-person Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()